In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

2025-03-07 15:04:18.938909: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741356259.973669   65370 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741356260.243939   65370 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-07 15:04:22.762482: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_frame = pd.read_pickle("../data/df/hpe-dnn-data.pkl")
data_frame.head()

,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,LEFT_ELBOW_x,LEFT_ELBOW_y,...,LEFT_THUMB_TIP_x,LEFT_THUMB_TIP_y,LEFT_THUMB_TIP_z,LEFT_INDEX_FINGER_MCP_x,LEFT_INDEX_FINGER_MCP_y,LEFT_INDEX_FINGER_MCP_z,LEFT_PINKY_MCP_x,LEFT_PINKY_MCP_y,LEFT_PINKY_MCP_z,technique
0,0.545526,0.634969,-0.015216,0.999819,0.517620,0.664082,-0.079635,0.999740,0.503807,0.597092,...,0.494822,0.510555,-0.000131,0.485555,0.515657,0.003581,0.487377,0.516973,-0.000188,4
1,0.544530,0.635610,0.036711,0.999777,0.518693,0.662950,-0.059934,0.999748,0.506267,0.596230,...,0.490785,0.513489,-0.002108,0.484118,0.519304,0.000906,0.494264,0.519330,-0.004506,4
2,0.542677,0.635603,0.017195,0.999752,0.517655,0.662962,-0.087714,0.999757,0.506260,0.595507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
3,0.541454,0.635623,0.010132,0.999707,0.517110,0.662973,-0.086193,0.999765,0.506208,0.595115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,0.540840,0.635446,0.005866,0.999702,0.516459,0.662955,-0.087612,0.999773,0.506049,0.595131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [3]:
data_frame.keys()

Index(['NOSE_x', 'NOSE_y', 'NOSE_z', 'NOSE_visibility', 'LEFT_SHOULDER_x',
       'LEFT_SHOULDER_y', 'LEFT_SHOULDER_z', 'LEFT_SHOULDER_visibility',
       'LEFT_ELBOW_x', 'LEFT_ELBOW_y', 'LEFT_ELBOW_z', 'LEFT_ELBOW_visibility',
       'LEFT_WRIST_x', 'LEFT_WRIST_y', 'LEFT_WRIST_z', 'LEFT_WRIST_visibility',
       'RIGHT_SHOULDER_x', 'RIGHT_SHOULDER_y', 'RIGHT_SHOULDER_z',
       'RIGHT_SHOULDER_visibility', 'RIGHT_ELBOW_x', 'RIGHT_ELBOW_y',
       'RIGHT_ELBOW_z', 'RIGHT_ELBOW_visibility', 'RIGHT_WRIST_x',
       'RIGHT_WRIST_y', 'RIGHT_WRIST_z', 'RIGHT_WRIST_visibility',
       'LEFT_HIP_x', 'LEFT_HIP_y', 'LEFT_HIP_z', 'LEFT_HIP_visibility',
       'LEFT_KNEE_x', 'LEFT_KNEE_y', 'LEFT_KNEE_z', 'LEFT_KNEE_visibility',
       'LEFT_ANKLE_x', 'LEFT_ANKLE_y', 'LEFT_ANKLE_z', 'LEFT_ANKLE_visibility',
       'LEFT_HEEL_x', 'LEFT_HEEL_y', 'LEFT_HEEL_z', 'LEFT_HEEL_visibility',
       'LEFT_FOOT_INDEX_x', 'LEFT_FOOT_INDEX_y', 'LEFT_FOOT_INDEX_z',
       'LEFT_FOOT_INDEX_visibility', 'RIGHT_HIP

In [4]:
train, val, test = np.split(data_frame.sample(frac=1), [int(0.7*len(data_frame)), int(0.85*len(data_frame))])

print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

561 training examples
120 validation examples
121 test examples


/mnt/c/Projects/climbing-technique-detector/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [31]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelBinarizer

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop("technique")

    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    df = pd.DataFrame(imp.fit_transform(df), columns=df.keys())
    df = {key: value.to_numpy()[:,tf.newaxis] for key, value in df.items()}

    encoder = LabelBinarizer()
    y = encoder.fit_transform(labels)

    ds = tf.data.Dataset.from_tensor_slices((dict(df), y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [6]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, _: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

def get_category_encoding_layer(name, dataset, max_tokens=None):
  
  index = layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, _: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))

In [32]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of Nose x-coordinates:', train_features['NOSE_x'])
print('A batch of techniques:', label_batch )

Every feature: ['NOSE_x', 'NOSE_y', 'NOSE_z', 'NOSE_visibility', 'LEFT_SHOULDER_x', 'LEFT_SHOULDER_y', 'LEFT_SHOULDER_z', 'LEFT_SHOULDER_visibility', 'LEFT_ELBOW_x', 'LEFT_ELBOW_y', 'LEFT_ELBOW_z', 'LEFT_ELBOW_visibility', 'LEFT_WRIST_x', 'LEFT_WRIST_y', 'LEFT_WRIST_z', 'LEFT_WRIST_visibility', 'RIGHT_SHOULDER_x', 'RIGHT_SHOULDER_y', 'RIGHT_SHOULDER_z', 'RIGHT_SHOULDER_visibility', 'RIGHT_ELBOW_x', 'RIGHT_ELBOW_y', 'RIGHT_ELBOW_z', 'RIGHT_ELBOW_visibility', 'RIGHT_WRIST_x', 'RIGHT_WRIST_y', 'RIGHT_WRIST_z', 'RIGHT_WRIST_visibility', 'LEFT_HIP_x', 'LEFT_HIP_y', 'LEFT_HIP_z', 'LEFT_HIP_visibility', 'LEFT_KNEE_x', 'LEFT_KNEE_y', 'LEFT_KNEE_z', 'LEFT_KNEE_visibility', 'LEFT_ANKLE_x', 'LEFT_ANKLE_y', 'LEFT_ANKLE_z', 'LEFT_ANKLE_visibility', 'LEFT_HEEL_x', 'LEFT_HEEL_y', 'LEFT_HEEL_z', 'LEFT_HEEL_visibility', 'LEFT_FOOT_INDEX_x', 'LEFT_FOOT_INDEX_y', 'LEFT_FOOT_INDEX_z', 'LEFT_FOOT_INDEX_visibility', 'RIGHT_HIP_x', 'RIGHT_HIP_y', 'RIGHT_HIP_z', 'RIGHT_HIP_visibility', 'RIGHT_KNEE_x', 'RIGHT_

In [33]:
batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [34]:
all_feature_labels = ['NOSE_x', 'NOSE_y', 'NOSE_z', 'NOSE_visibility', 'LEFT_SHOULDER_x',
       'LEFT_SHOULDER_y', 'LEFT_SHOULDER_z', 'LEFT_SHOULDER_visibility',
       'LEFT_ELBOW_x', 'LEFT_ELBOW_y', 'LEFT_ELBOW_z', 'LEFT_ELBOW_visibility',
       'LEFT_WRIST_x', 'LEFT_WRIST_y', 'LEFT_WRIST_z', 'LEFT_WRIST_visibility',
       'RIGHT_SHOULDER_x', 'RIGHT_SHOULDER_y', 'RIGHT_SHOULDER_z',
       'RIGHT_SHOULDER_visibility', 'RIGHT_ELBOW_x', 'RIGHT_ELBOW_y',
       'RIGHT_ELBOW_z', 'RIGHT_ELBOW_visibility', 'RIGHT_WRIST_x',
       'RIGHT_WRIST_y', 'RIGHT_WRIST_z', 'RIGHT_WRIST_visibility',
       'LEFT_HIP_x', 'LEFT_HIP_y', 'LEFT_HIP_z', 'LEFT_HIP_visibility',
       'LEFT_KNEE_x', 'LEFT_KNEE_y', 'LEFT_KNEE_z', 'LEFT_KNEE_visibility',
       'LEFT_ANKLE_x', 'LEFT_ANKLE_y', 'LEFT_ANKLE_z', 'LEFT_ANKLE_visibility',
       'LEFT_HEEL_x', 'LEFT_HEEL_y', 'LEFT_HEEL_z', 'LEFT_HEEL_visibility',
       'LEFT_FOOT_INDEX_x', 'LEFT_FOOT_INDEX_y', 'LEFT_FOOT_INDEX_z',
       'LEFT_FOOT_INDEX_visibility', 'RIGHT_HIP_x', 'RIGHT_HIP_y',
       'RIGHT_HIP_z', 'RIGHT_HIP_visibility', 'RIGHT_KNEE_x', 'RIGHT_KNEE_y',
       'RIGHT_KNEE_z', 'RIGHT_KNEE_visibility', 'RIGHT_ANKLE_x',
       'RIGHT_ANKLE_y', 'RIGHT_ANKLE_z', 'RIGHT_ANKLE_visibility',
       'RIGHT_HEEL_x', 'RIGHT_HEEL_y', 'RIGHT_HEEL_z', 'RIGHT_HEEL_visibility',
       'RIGHT_FOOT_INDEX_x', 'RIGHT_FOOT_INDEX_y', 'RIGHT_FOOT_INDEX_z',
       'RIGHT_FOOT_INDEX_visibility', 'RIGHT_THUMB_MCP_x', 'RIGHT_THUMB_MCP_y',
       'RIGHT_THUMB_MCP_z', 'RIGHT_THUMB_IP_x', 'RIGHT_THUMB_IP_y',
       'RIGHT_THUMB_IP_z', 'RIGHT_THUMB_TIP_x', 'RIGHT_THUMB_TIP_y',
       'RIGHT_THUMB_TIP_z', 'RIGHT_INDEX_FINGER_MCP_x',
       'RIGHT_INDEX_FINGER_MCP_y', 'RIGHT_INDEX_FINGER_MCP_z',
       'RIGHT_PINKY_MCP_x', 'RIGHT_PINKY_MCP_y', 'RIGHT_PINKY_MCP_z',
       'LEFT_THUMB_MCP_x', 'LEFT_THUMB_MCP_y', 'LEFT_THUMB_MCP_z',
       'LEFT_THUMB_IP_x', 'LEFT_THUMB_IP_y', 'LEFT_THUMB_IP_z',
       'LEFT_THUMB_TIP_x', 'LEFT_THUMB_TIP_y', 'LEFT_THUMB_TIP_z',
       'LEFT_INDEX_FINGER_MCP_x', 'LEFT_INDEX_FINGER_MCP_y',
       'LEFT_INDEX_FINGER_MCP_z', 'LEFT_PINKY_MCP_x', 'LEFT_PINKY_MCP_y',
       'LEFT_PINKY_MCP_z']

all_inputs = {}
encoded_feature = []

for header in all_feature_labels:
    numerical_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    normalized_numerical_col = normalization_layer(numerical_col)
    all_inputs[header] = numerical_col
    encoded_feature.append(normalized_numerical_col)

2025-03-07 15:22:41.490570: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [35]:
encoded_feature

[<KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_108>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_109>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_110>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_111>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_112>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_113>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_114>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_115>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_116>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_117>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_118>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tenso

In [36]:
from keras import Model

all_features = layers.concatenate(encoded_feature)
x = layers.Dense(256, activation="relu")(all_features)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.1)(x)
output = layers.Dense(4, activation="softmax")(x)

model = Model(all_inputs, output)


In [40]:
from keras import losses

model.compile(optimizer='adam', loss=losses.BinaryCrossentropy(), metrics=["accuracy"], run_eagerly=True)

In [38]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, rankdir="LR")

KeyboardInterrupt: 

In [41]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 244ms/step - accuracy: 0.8206 - loss: 0.2972 - val_accuracy: 0.8167 - val_loss: 0.2648
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step - accuracy: 0.8361 - loss: 0.2646 - val_accuracy: 0.8500 - val_loss: 0.2403
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 419ms/step - accuracy: 0.8525 - loss: 0.2401 - val_accuracy: 0.8667 - val_loss: 0.2122
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step - accuracy: 0.8620 - loss: 0.2146 - val_accuracy: 0.8750 - val_loss: 0.1903
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 235ms/step - accuracy: 0.8804 - loss: 0.1952 - val_accuracy: 0.8750 - val_loss: 0.1772
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.8875 - loss: 0.1877 - val_accuracy: 0.8917 - val_loss: 0.1675
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 245ms/step - accuracy: 0.8793 - loss: 0.1701 - val_accuracy: 0.9083 - val_loss: 0.1620
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 245ms/step - accuracy: 0.9030 - loss: 0.1582 - val_accuracy: 0.9083 - val_loss: 0.

In [42]:
results = model.evaluate(test_ds, return_dict=True)
print(results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.8595 - loss: 0.1859
{'accuracy': 0.8595041036605835, 'loss': 0.185917928814888}
